<a href="https://colab.research.google.com/github/raamav/Sentiment-Analysis-UserReviews/blob/master/IMDB_Reviews_DeepLearning_(v2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB Reviews

**IMDB Reviews** is one of the best known datasets for sentiment analysis. It comprises a set of 50,000 highly polarized reviews of movies which are tagged with *positive* and *negative* sentiment labels.

The 50,000 reviews are categorized into training and test sets (25,000 reviews each). These reviews are full-length and some of them are several paragraphs long.

I've worked on this dataset previously and have got a validation accuracy of 80%.

**In this attempt, the objective is to get an accuracy of 85% or better.**

In [1]:
# STANDARD IMPORTS

import tensorflow as tf
from tensorflow import keras
import numpy as np


tf.__version__


'2.2.0-rc2'

In [2]:
import tensorflow_datasets as tfds

imdb,info = tfds.load("imdb_reviews", with_info = True, as_supervised = True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete64QC7T/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete64QC7T/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete64QC7T/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
train_data = imdb['train']
test_data = imdb['test']


In [4]:
print(type(train_data))

<class 'tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter'>


In [0]:
# GETTING FEATURES AND LABELS FOR TRAINING AND TEST DATASETS

X_train_l = [] # sentences
y_train_l = [] # sentiment 

X_test_l = []
y_test_l = []

for s,l in train_data:
  X_train_l.append(str(s.numpy()))
  y_train_l.append(l.numpy)


for s,l in test_data:
  X_test_l.append(str(s.numpy()))
  y_test_l.append(l.numpy)



In [6]:
# DATATYPES AND SIZES OF THE DATASETS


print('/n Training Data and Label, Type {} : {}' .format(type(X_train_l) , type(y_train_l)))
print('/n Training Data and Label, Sizes {} : {}' .format(len(X_train_l) , len(y_train_l)))

/n Training Data and Label, Type <class 'list'> : <class 'list'>
/n Training Data and Label, Sizes 25000 : 25000


In [0]:
# CONVERTING EVERYTHING TO NUMPY ARRAYS
# MAJOR RELIEF POINT AS ITS TRICKY WORKING WITH TENSORS

import numpy as np

X_train = np.array(X_train_l)
y_train = np.array(y_train_l)

X_test = np.array(X_test_l)
y_test = np.array(y_test_l)

In [8]:
print('/n Training Data and Label, Type {} : {}' .format(type(X_train) , type(y_train)))
print('/n Training Data and Label, Sizes {} : {}' .format((X_train.shape) , (y_train.shape)))

/n Training Data and Label, Type <class 'numpy.ndarray'> : <class 'numpy.ndarray'>
/n Training Data and Label, Sizes (25000,) : (25000,)
